#### Convolution CNN for leaning images and processing - using high-level Keras preprocessing utilities and layers to read a directory of images on disk.

https://www.tensorflow.org/tutorials/load_data/images

#### deep_learn2.ipynb - was run for 1 year images.

In [68]:
import warnings
warnings.filterwarnings('ignore')

In [28]:
import numpy as np
import pandas as pd
import os
import os.path
import PIL
from PIL import Image
import PIL.Image
import tensorflow as tf
#import tensorflow_datasets as tfds
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
import datetime

In [8]:
print(tf.__version__)

2.5.0


#### Dataset - directory
candlesticks_together

##### Define path for dataset

In [68]:
import pathlib
image_dir = pathlib.Path('candlesticks_together/')
#data_test_dir = pathlib.Path('candlesticks_test')

### Create File DataFrame

In [69]:
filepaths = pd.Series(list(image_dir.glob(r'**/*.png')), name='Filepath').astype(str)

In [70]:
filepaths

0     candlesticks_together\2020-07-12.png
1     candlesticks_together\2020-07-19.png
2     candlesticks_together\2020-07-26.png
3     candlesticks_together\2020-08-02.png
4     candlesticks_together\2020-08-09.png
5     candlesticks_together\2020-08-16.png
6     candlesticks_together\2020-08-23.png
7     candlesticks_together\2020-08-30.png
8     candlesticks_together\2020-09-06.png
9     candlesticks_together\2020-09-13.png
10    candlesticks_together\2020-09-20.png
11    candlesticks_together\2020-09-27.png
12    candlesticks_together\2020-10-04.png
13    candlesticks_together\2020-10-11.png
14    candlesticks_together\2020-10-18.png
15    candlesticks_together\2020-10-25.png
16    candlesticks_together\2020-11-01.png
17    candlesticks_together\2020-11-08.png
18    candlesticks_together\2020-11-15.png
19    candlesticks_together\2020-11-22.png
20    candlesticks_together\2020-11-29.png
21    candlesticks_together\2020-12-06.png
22    candlesticks_together\2020-12-13.png
23    candl

In [71]:
signal_df = pd.read_csv('signal.csv')  

In [92]:
signal_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52 entries, 0 to 51
Data columns (total 2 columns):
Date      52 non-null object
Signal    52 non-null int64
dtypes: int64(1), object(1)
memory usage: 912.0+ bytes


In [93]:
signal_df['Date']= pd.to_datetime(signal_df['Date'])

In [94]:
signal_df.set_index('Date', inplace = True)

In [95]:
signal_df.head()

,Signal
Date,
2020-07-12,0
2020-07-19,1
2020-07-26,1
2020-08-02,1
2020-08-09,1


In [96]:
signal_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 52 entries, 2020-07-12 to 2021-07-04
Data columns (total 1 columns):
Signal    52 non-null int64
dtypes: int64(1)
memory usage: 832.0 bytes


In [74]:
filepaths[0]

'candlesticks_together\\2020-07-12.png'

In [75]:
date_list = []
for i in filepaths:
    #print(i)
    dtstr = os.path.split(i)[1].split(".")[0]
    #print(dtstr)
    #print(datetime.datetime.strptime(dtstr, '%Y-%m-%d').date())
    date_list.append(datetime.datetime.strptime(dtstr, '%Y-%m-%d').date())
    

In [76]:
date_list

[datetime.date(2020, 7, 12),
 datetime.date(2020, 7, 19),
 datetime.date(2020, 7, 26),
 datetime.date(2020, 8, 2),
 datetime.date(2020, 8, 9),
 datetime.date(2020, 8, 16),
 datetime.date(2020, 8, 23),
 datetime.date(2020, 8, 30),
 datetime.date(2020, 9, 6),
 datetime.date(2020, 9, 13),
 datetime.date(2020, 9, 20),
 datetime.date(2020, 9, 27),
 datetime.date(2020, 10, 4),
 datetime.date(2020, 10, 11),
 datetime.date(2020, 10, 18),
 datetime.date(2020, 10, 25),
 datetime.date(2020, 11, 1),
 datetime.date(2020, 11, 8),
 datetime.date(2020, 11, 15),
 datetime.date(2020, 11, 22),
 datetime.date(2020, 11, 29),
 datetime.date(2020, 12, 6),
 datetime.date(2020, 12, 13),
 datetime.date(2020, 12, 20),
 datetime.date(2020, 12, 27),
 datetime.date(2021, 1, 3),
 datetime.date(2021, 1, 10),
 datetime.date(2021, 1, 17),
 datetime.date(2021, 1, 24),
 datetime.date(2021, 1, 31),
 datetime.date(2021, 2, 7),
 datetime.date(2021, 2, 14),
 datetime.date(2021, 2, 21),
 datetime.date(2021, 2, 28),
 datetime.

In [37]:
fn = datetime.datetime.strptime(os.path.split(filepaths[0])[1].split(".")[0], '%Y-%m-%d').date()

In [38]:
fn

datetime.date(2020, 7, 19)

In [22]:
filename = os.path.split(filepaths[0])[1]

In [23]:
filename

'2020-07-19.png'

In [25]:
parts = filename.split(".")

In [30]:
parts[0]

'2020-07-19'

In [32]:
image_date=datetime.datetime.strptime(parts[0], '%Y-%m-%d').date()

In [33]:
image_date

datetime.date(2020, 7, 19)

In [34]:
type(image_date)

datetime.date

In [85]:
images_df = pd.DataFrame()
images_df['Date'] = date_list 
images_df['Filepath'] =  filepaths

In [86]:
images_df['Date']= pd.to_datetime(images_df['Date'])

In [87]:
images_df.head()

,Date,Filepath
0,2020-07-12,candlesticks_together\2020-07-12.png
1,2020-07-19,candlesticks_together\2020-07-19.png
2,2020-07-26,candlesticks_together\2020-07-26.png
3,2020-08-02,candlesticks_together\2020-08-02.png
4,2020-08-09,candlesticks_together\2020-08-09.png


In [88]:
images_df.set_index('Date', inplace=True)

In [89]:
images_df.head()

,Filepath
Date,
2020-07-12,candlesticks_together\2020-07-12.png
2020-07-19,candlesticks_together\2020-07-19.png
2020-07-26,candlesticks_together\2020-07-26.png
2020-08-02,candlesticks_together\2020-08-02.png
2020-08-09,candlesticks_together\2020-08-09.png


In [90]:
images_df.index

DatetimeIndex(['2020-07-12', '2020-07-19', '2020-07-26', '2020-08-02',
               '2020-08-09', '2020-08-16', '2020-08-23', '2020-08-30',
               '2020-09-06', '2020-09-13', '2020-09-20', '2020-09-27',
               '2020-10-04', '2020-10-11', '2020-10-18', '2020-10-25',
               '2020-11-01', '2020-11-08', '2020-11-15', '2020-11-22',
               '2020-11-29', '2020-12-06', '2020-12-13', '2020-12-20',
               '2020-12-27', '2021-01-03', '2021-01-10', '2021-01-17',
               '2021-01-24', '2021-01-31', '2021-02-07', '2021-02-14',
               '2021-02-21', '2021-02-28', '2021-03-07', '2021-03-14',
               '2021-03-21', '2021-03-28', '2021-04-04', '2021-04-11',
               '2021-04-18', '2021-04-25', '2021-05-02', '2021-05-09',
               '2021-05-16', '2021-05-23', '2021-05-30', '2021-06-06',
               '2021-06-13', '2021-06-20', '2021-06-27', '2021-07-04'],
              dtype='datetime64[ns]', name='Date', freq=None)

In [105]:
images = pd.concat([images_df,signal_df], join='inner', axis=1)

In [106]:
images.head(10)

,Filepath,Signal
Date,,
2020-07-12,candlesticks_together\2020-07-12.png,0
2020-07-19,candlesticks_together\2020-07-19.png,1
2020-07-26,candlesticks_together\2020-07-26.png,1
2020-08-02,candlesticks_together\2020-08-02.png,1
2020-08-09,candlesticks_together\2020-08-09.png,1
2020-08-16,candlesticks_together\2020-08-16.png,0
2020-08-23,candlesticks_together\2020-08-23.png,1
2020-08-30,candlesticks_together\2020-08-30.png,1
2020-09-06,candlesticks_together\2020-09-06.png,0


In [107]:
# Let's only use 40 images to speed up training time
image_df = images.sample(40, random_state=1).reset_index(drop=True)
train_df, test_df = train_test_split(image_df, train_size=0.7, shuffle=True, random_state=1)

In [108]:
train_df.head()

,Filepath,Signal
30,candlesticks_together\2020-10-18.png,0
36,candlesticks_together\2020-11-29.png,1
33,candlesticks_together\2021-05-30.png,0
28,candlesticks_together\2020-08-09.png,1
4,candlesticks_together\2021-02-07.png,1


In [109]:
test_df.head()

,Filepath,Signal
2,candlesticks_together\2021-05-02.png,1
31,candlesticks_together\2021-01-31.png,1
3,candlesticks_together\2021-02-21.png,0
21,candlesticks_together\2020-10-11.png,1
27,candlesticks_together\2020-12-20.png,1


### Loading Images

In [110]:
train_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

test_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255
)

In [111]:
train_df

,Filepath,Signal
30,candlesticks_together\2020-10-18.png,0
36,candlesticks_together\2020-11-29.png,1
33,candlesticks_together\2021-05-30.png,0
28,candlesticks_together\2020-08-09.png,1
4,candlesticks_together\2021-02-07.png,1
14,candlesticks_together\2020-12-27.png,1
10,candlesticks_together\2020-09-20.png,1
35,candlesticks_together\2020-11-15.png,1
23,candlesticks_together\2021-01-10.png,0
24,candlesticks_together\2020-11-08.png,0


In [112]:
train_images = train_generator.flow_from_dataframe(
    dataframe=train_df,
    x_col='Filepath',
    y_col='Signal',
    target_size=(120, 120),
    color_mode='rgb',
    class_mode='raw',
    batch_size=32,
    shuffle=True,
    seed=42,
    subset='training'
)

val_images = train_generator.flow_from_dataframe(
    dataframe=train_df,
    x_col='Filepath',
    y_col='Signal',
    target_size=(120, 120),
    color_mode='rgb',
    class_mode='raw',
    batch_size=32,
    shuffle=True,
    seed=42,
    subset='validation'
)

test_images = test_generator.flow_from_dataframe(
    dataframe=test_df,
    x_col='Filepath',
    y_col='Signal',
    target_size=(120, 120),
    color_mode='rgb',
    class_mode='raw',
    batch_size=32,
    shuffle=False
)

Found 23 validated image filenames.
Found 5 validated image filenames.
Found 12 validated image filenames.


### Training

In [113]:
inputs = tf.keras.Input(shape=(120, 120, 3))
x = tf.keras.layers.Conv2D(filters=16, kernel_size=(3, 3), activation='relu')(inputs)
x = tf.keras.layers.MaxPool2D()(x)
x = tf.keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu')(x)
x = tf.keras.layers.MaxPool2D()(x)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dense(64, activation='relu')(x)
x = tf.keras.layers.Dense(64, activation='relu')(x)
outputs = tf.keras.layers.Dense(1, activation='linear')(x)

model = tf.keras.Model(inputs=inputs, outputs=outputs)

model.compile(
    optimizer='adam',
    loss='mse'
)

history = model.fit(
    train_images,
    validation_data=val_images,
    epochs=100,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=5,
            restore_best_weights=True
        )
    ]
)

Epoch 1/100
1/1 [==============================] - 3s 3s/step - loss: 0.6479 - val_loss: 0.5834
Epoch 2/100
1/1 [==============================] - 0s 379ms/step - loss: 0.6338 - val_loss: 0.5703
Epoch 3/100
1/1 [==============================] - 1s 596ms/step - loss: 0.6194 - val_loss: 0.5572
Epoch 4/100
1/1 [==============================] - 0s 320ms/step - loss: 0.6051 - val_loss: 0.5431
Epoch 5/100
1/1 [==============================] - 0s 347ms/step - loss: 0.5895 - val_loss: 0.5280
Epoch 6/100
1/1 [==============================] - 0s 337ms/step - loss: 0.5725 - val_loss: 0.5115
Epoch 7/100
1/1 [==============================] - 1s 701ms/step - loss: 0.5541 - val_loss: 0.4938
Epoch 8/100
1/1 [==============================] - 1s 555ms/step - loss: 0.5341 - val_loss: 0.4748
Epoch 9/100
1/1 [==============================] - 0s 430ms/step - loss: 0.5126 - val_loss: 0.4546
Epoch 10/100
1/1 [==============================] - 0s 341ms/step - loss: 0.4896 - val_loss: 0.4328
Epoch 11/100

### Results

In [115]:
predicted_signals = np.squeeze(model.predict(test_images))
true_signals = test_images.labels

rmse = np.sqrt(model.evaluate(test_images, verbose=0))
print("     Test RMSE: {:.5f}".format(rmse))

r2 = r2_score(true_signals, predicted_signals)
print("Test R^2 Score: {:.5f}".format(r2))

     Test RMSE: 0.47129
Test R^2 Score: -0.59920


In [117]:
null_rmse = np.sqrt(np.sum((true_signals - np.mean(true_signals))**2) / len(true_signals))
print("Null/Baseline Model Test RMSE: {:.5f}".format(null_rmse))

Null/Baseline Model Test RMSE: 0.37268


## We can see that our model is not even better than the null/baseline model (predict mean every time)!

In [119]:
true_signals

array([1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1], dtype=int64)

In [120]:
predicted_signals

array([0.5338014 , 0.4910986 , 0.48627794, 0.6038471 , 0.50324136,
       0.5790675 , 0.50132656, 0.5709462 , 0.5417198 , 0.56894696,
       0.4593557 , 0.59095013], dtype=float32)

In [121]:
len(true_signals)

12

In [122]:
len(predicted_signals)

12

In [125]:
final_predictions_df = pd.DataFrame()
final_predictions_df['Actual'] = true_signals
final_predictions_df['Predicted'] =  predicted_signals

In [126]:
final_predictions_df

,Actual,Predicted
0,1,0.533801
1,1,0.491099
2,0,0.486278
3,1,0.603847
4,1,0.503241
5,1,0.579068
6,1,0.501327
7,1,0.570946
8,0,0.541720
9,1,0.568947
